### 회귀

#### K-Fold

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

In [2]:
#데이터 수집
x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6,12],
    [9,2],
    [6,10],
    [2,4]
])

In [3]:
kf = KFold(n_splits=5)
#KFold 적용,데이터셋을 5개로 나눔

In [4]:
#교차검증의 train 데이터셋과 test 데이터셋을 확인하기 위한 반복문
for train_index,test_index in kf.split(x_data):
    print('train_index:',train_index)
    print('test_index:',test_index)

train_index: [ 3  4  5  6  7  8  9 10 11]
test_index: [0 1 2]
train_index: [ 0  1  2  6  7  8  9 10 11]
test_index: [3 4 5]
train_index: [ 0  1  2  3  4  5  8  9 10 11]
test_index: [6 7]
train_index: [ 0  1  2  3  4  5  6  7 10 11]
test_index: [8 9]
train_index: [0 1 2 3 4 5 6 7 8 9]
test_index: [10 11]


### K-Fold 교차검증 -> 보통 회귀 문제에 사용됨
- 학습데이터와 테스트 데이터를 k개의 세트로 나누어 검증하는 방법
- 데이터셋이 굉장히 적을 때 훈련 데이터를 어떻게든 최대한 늘려보려고 사용하는 것
- 여러개의 훈련테스트 짝으로 검증

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [6]:
#데이터 수집
x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6,12],
    [9,2],
    [6,10],
    [2,4]
])

y_data = np.array([3,5,7,10,12,7,13,13,12,13,12,6])

In [7]:
lr = LinearRegression()

In [8]:
#교차검증의 정확도를 보기 위한 for문
train_scores = []
test_scores = []
kf = KFold(n_splits=5) #데이터 5개로 쪼갬

for train_index,test_index in kf.split(x_data):
    x_train = np.array(x_data)[train_index]
    y_train = np.array(y_data)[train_index]
    x_test = np.array(x_data)[test_index]
    y_test = np.array(y_data)[test_index]
    
    lr= LinearRegression() #모델선정
    lr.fit(x_train,y_train) #모델 학습
    #훈련데이터
    score = lr.score(x_train,y_train)
    train_scores.append(score) #r2_score
    
    #평가데이터
    score_test = lr.score(x_test,y_test)
    test_scores.append(score_test) #r2_score

In [9]:
train_scores

[0.9522707858769932,
 0.9469593697441799,
 0.9446524178499608,
 0.9232432525564045,
 0.9166499001004778]

In [10]:
test_scores

[-1.1475590101753324,
 0.56847222331606,
 0.0,
 -11.7747639790487,
 0.9602035173350366]

In [11]:
print(np.array(train_scores).mean())
#train데이터의 r2_score 평균

0.9367551452256032


In [12]:
print(np.array(test_scores).mean())
#test 데이터의 r2_score 평균
# -값나옴, 예측이 잘 안됨.

-2.278729449714587


In [13]:
lr.fit(x_data,y_data)
x_test=np.array([
    [4,6]
])
y_predict = lr.predict(x_test)
#새로운 데이터 넣고 그 데이터에 대한 예측값 확인

In [14]:
y_predict

array([8.27950438])

### 계층별 K-겹 교차검증(Stratified K-Fold Cross Validation)
- 분류모델에 적용
- k-겹 교차검증은 k-fold가 원본 데이터 집합의 레이블 분포를 학습 및 검증 데이터 세트에 제대로 분배하지 못하는 문제가 있는데, 레이블에 속성값의 개수를 골고루 넣어줌

- 계층적 교차 검증은 K-fold의 향상된 버전입니다. 이 방법은 폴드를 나눌 때 각 폴드간의 클래스 비율을 전체 훈련 세트의 클래스 비율과 일치하게 나눕니다. 이것은 결과적으로 모델이 좀더 나은 편향과 분산을 가지게 합니다.

In [15]:
from sklearn.model_selection import cross_val_score, StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [16]:
iris = load_iris()
#데이터 수집

In [17]:
lr = LogisticRegression()
#모델선정

In [18]:
skf = StratifiedKFold(n_splits=5,random_state=3,shuffle=True)
#계층별 K-fold 교차검증, 5개의 데이터로 데이터 나눔, 3번 시드
#shuffle = True :데이터를 무작위로 섞음

In [19]:
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [20]:
x = iris.data
y = iris.target

In [21]:
#교차검증의 정확도를 확인하기 위한 반복문 작성 
idx_iter = 0 #반복수
cv_accuracy = []

for train_index,test_index in skf.split(x,y):
    x_train,x_test = x[train_index],x[test_index]
    y_train,y_test = y[train_index],y[test_index]
    
    #모델 학습(모델 계속 선정하면 오류날수도 있어서 위에 코드를 사용)
    lr.fit(x_train,y_train) 
    
    pred = lr.predict(x_test) #예측값 산출
    
    idx_iter += 1 #반복수 1 추가
    accuracy = np.round(accuracy_score(y_test,pred),4)
    #np.round(data, n) = 데이터를 소수 n번째저라까지 나오게 반올림
    train_size = x_train.shape[0] #x_train의 행의 개수
    test_size = x_test.shape[0] #x_test의 행의 개수
    
    print('{0}번째 교차 검증 정확도: {1} \n 학습 데이터 크기: {2} \n 검증 데이터 크기 : {3}'.format(idx_iter,
                                                                             accuracy,
                                                                             train_size,
                                                                             test_size))
    #.format()을 쓸때 괄호안에 먼저 나온것부터 {0}번 중괄호에 들어가는 식
    # 들어가는 순서
    
    cv_accuracy.append(accuracy)

1번째 교차 검증 정확도: 1.0 
 학습 데이터 크기: 120 
 검증 데이터 크기 : 30
2번째 교차 검증 정확도: 0.9667 
 학습 데이터 크기: 120 
 검증 데이터 크기 : 30
3번째 교차 검증 정확도: 0.9 
 학습 데이터 크기: 120 
 검증 데이터 크기 : 30
4번째 교차 검증 정확도: 1.0 
 학습 데이터 크기: 120 
 검증 데이터 크기 : 30
5번째 교차 검증 정확도: 0.9667 
 학습 데이터 크기: 120 
 검증 데이터 크기 : 30


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [22]:
print('교차 검증별 정확도 :',np.round(cv_accuracy,4))
print('평균 검증 정확도 :',np.mean(cv_accuracy))

교차 검증별 정확도 : [1.     0.9667 0.9    1.     0.9667]
평균 검증 정확도 : 0.96668


<교차 검증의 장점>
1. test set에 데이터가 최소 한 번씩 들어가기 때문에 모델이 더 잘 일반화 됨
2. 분할을 한 번 했을 때보다 데이터를 더 효과적으로 사용해서 더 정확한 모델을 만들 수 있다

<교차 검증의 단점>
1. 연산 비용이 늘어남(k개의 모델을 만들어야 하므로 데이터를 한 번 나눴을 때에 비해서 느리다.)